In [2]:
# run this for google colab:
from google.colab import drive
drive.mount('/content/drive')

# locate correct directory
%cd drive/MyDrive/EE269\ Final\ Project/mel-train/
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/EE269 Final Project/mel-train/'
/content/drive/MyDrive/EE269 Final Project/mel-train
cldm/                       make_cnet_dataset.ipynb  test-data/
cnet_riff_add_control.py    make_cnet_dataset.py     test_logger.ipynb
cnet_riff_dataset.py        models/                  test_sample_target.wav
cnet_riff_preprocessing.py  prompt_labels.json       train-data/
cnet_riff_training.py       __pycache__/             train-data-old/
config.py                   README.md                train.sh
demo_audio_issue.ipynb      riffusion/               uncorrupt_sources.ipynb
envs/                       riffusion-cnet-v2/       upload_checkpoints.py
gen_data.sh                 sample_cnet_riff.ipynb   utils/
get_dataset_sample.py       samples/
ldm/                        share.py


In [5]:
# get training data
!git clone https://github.com/zachary-shah/train-data.git

# load model from huggingface
!git lfs install
!git clone https://huggingface.co/zachary-shah/riffusion-cnet-v2

Cloning into 'train-data'...
remote: Enumerating objects: 19508, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 19508 (delta 0), reused 3 (delta 0), pack-reused 19503
Receiving objects: 100% (19508/19508), 1.44 GiB | 21.03 MiB/s, done.
Updating files: 100% (19739/19739), done.


In [6]:
# install dependencies
!pip install einops
!pip install omegaconf
!pip install pytorch_lightning
!pip install transformers
!pip install open-clip-torch
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 2.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=d83a4b6ec17faa3ae981d028a58b193ae523a6af8c7cd199ffbccebd22208835
  Stored in directory: /root/.cache/pip/wheels/23/cf/80/f3efa822e6ab23277902ee9165fe772eeb1dfb8014f359020a
Successfully built antlr4-python3-runtime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.4 MB/s eta 0:00:

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import torch 
import einops 
from pytorch_lightning import seed_everything
import cv2
import os
from PIL import Image
import numpy as np
import random 

from riffusion.spectrogram_image_converter import SpectrogramImageConverter
from riffusion.spectrogram_converter import SpectrogramConverter
from riffusion.spectrogram_params import SpectrogramParams

from cldm.ddim_hacked import DDIMSampler
from cnet_riff_dataset import CnetRiffDataset
from cldm.model import create_model, load_state_dict

In [6]:
# load model into pytorch 
mdl_path = "riffusion-cnet-v2/final_checkpoint.ckpt"
model = create_model('./models/cldm_v15.yaml').cpu()
model.load_state_dict(load_state_dict(mdl_path, location='cuda'))
model = model.cuda()
ddim_sampler = DDIMSampler(model)

No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.19.self_attn.k_proj.bias', 'vision_model.encoder.layers.18.mlp.fc2.weight', 'vision_model.encoder.layers.2.layer_norm2.weight', 'vision_model.encoder.layers.4.mlp.fc1.bias', 'vision_model.encoder.layers.8.layer_norm2.bias', 'vision_model.encoder.layers.8.self_attn.v_proj.weight', 'vision_model.encoder.layers.2.self_attn.out_proj.weight', 'vision_model.encoder.layers.18.mlp.fc1.weight', 'vision_model.encoder.layers.4.layer_norm2.weight', 'vision_model.encoder.layers.19.mlp.fc1.weight', 'vision_model.encoder.layers.19.mlp.fc1.bias', 'vision_model.encoder.layers.21.mlp.fc2.bias', 'vision_model.encoder.layers.20.mlp.fc1.bias', 'vision_model.encoder.layers.3.self_attn.k_proj.bias', 'vision_model.encoder.layers.7.self_attn.out_proj.bias', 'vision_model.encoder.layers.14.self_attn.v_proj.bias', 'vision_model.encoder.layers.16.layer_norm1.weight', '

Loaded model config from [./models/cldm_v15.yaml]
Loaded state_dict from [riffusion-cnet-v2/final_checkpoint.ckpt]


In [10]:
# image helper functions
def HWC3(x):
    assert x.dtype == np.uint8
    if x.ndim == 2:
        x = x[:, :, None]
    assert x.ndim == 3
    H, W, C = x.shape
    assert C == 1 or C == 3 or C == 4
    if C == 3:
        return x
    if C == 1:
        return np.concatenate([x, x, x], axis=2)
    if C == 4:
        color = x[:, :, 0:3].astype(np.float32)
        alpha = x[:, :, 3:4].astype(np.float32) / 255.0
        y = color * alpha + 255.0 * (1.0 - alpha)
        y = y.clip(0, 255).astype(np.uint8)
        return y

def resize_image(input_image, resolution):
    H, W, C = input_image.shape
    H = float(H)
    W = float(W)
    k = float(resolution) / min(H, W)
    H *= k
    W *= k
    H = int(np.round(H / 64.0)) * 64
    W = int(np.round(W / 64.0)) * 64
    img = cv2.resize(input_image, (W, H), interpolation=cv2.INTER_LANCZOS4 if k > 1 else cv2.INTER_AREA)
    return img

In [12]:
# sample the model
def sample_ddim(control, prompt, model,
                guess_mode = False,
                strength = 1,
                scale = 9,
                eta = 0,
                num_samples = 1,
                ddim_steps = 50,
                seed = -1,
                image_resolution = 512):

    with torch.no_grad():
        # preprocessing
        control = resize_image(HWC3(control), image_resolution)
        H, W, C = control.shape
        control = torch.from_numpy(test_dataset[i]['hint']).float().cuda()
        control = torch.stack([control for _ in range(num_samples)], dim=0)
        control = einops.rearrange(control, 'b h w c -> b c h w').clone()

        if seed == -1:
            seed = random.randint(0, 65535)
        seed_everything(seed)

        # to save memory
        model.low_vram_shift(is_diffusing=False)

        cond = {"c_concat": [control], "c_crossattn": [model.get_learned_conditioning([prompt] * num_samples)]}
        un_cond = {"c_concat": None if guess_mode else [control], "c_crossattn": [model.get_learned_conditioning([""] * num_samples)]}
        shape = (4, H // 8, W // 8)

        # to save memory
        model.low_vram_shift(is_diffusing=True)

        # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
        model.control_scales = [strength * (0.825 ** float(12 - i)) for i in range(13)] if guess_mode else ([strength] * 13)  
        
        samples, intermediates = ddim_sampler.sample(ddim_steps, num_samples,
                                                        shape, cond, verbose=False, eta=eta,
                                                        unconditional_guidance_scale=scale,
                                                        unconditional_conditioning=un_cond)

        # to save memory
        model.low_vram_shift(is_diffusing=False)

        x_samples = model.decode_first_stage(samples)
        x_samples = (einops.rearrange(x_samples, 'b c h w -> b h w c') * 127.5 + 127.5).cpu().numpy().clip(0, 255).astype(np.uint8)

        results = [x_samples[i] for i in range(num_samples)]

        return results, intermediates

In [26]:
# load training dataset
test_dataset = CnetRiffDataset("train-data/")
print(f"Len dataset: {len(test_dataset)}")

os.makedirs("samples", exist_ok=True)
img_converter_to_audio = SpectrogramImageConverter(SpectrogramParams(sample_rate=44100, min_frequency=0, max_frequency=10000))

for (i,item) in enumerate(test_dataset):
    if np.mod(i, 100) == 0:
        # sample the model four times using conditioned edges, prompt
        results, _ = sample_ddim(item['hint'], item['txt'], model, num_samples=4)

        # save each sample spectrogram and convert to audio and save that too
        for (k, sample) in enumerate(results):
            cv2.imwrite(os.path.join("samples/",f"{item['txt']}_samp_{k}.png"), sample)

            sample_img = Image.open(os.path.join("samples/",f"{item['txt']}_samp_{k}.png"))
            out_audio_recon = img_converter_to_audio.audio_from_spectrogram_image(sample_img, apply_filters=True).set_channels(2)
            out_audio_recon.export(os.path.join("samples/",f"{item['txt']}_samp_{k}.wav"), format="wav")

        # save target spec and audio too
        target = (item['jpg'] + 1) / 2 * 255
        cv2.imwrite(os.path.join("samples/",f"{item['txt']}_target.png"), target)
        target_img = Image.open(os.path.join("samples/",f"{item['txt']}_target.png")) 

        out_audio_recon = img_converter_to_audio.audio_from_spectrogram_image(target_img, apply_filters=True).set_channels(2)
        out_audio_recon.export(os.path.join("samples/",f"{item['txt']}_target.wav"), format="wav")

    # # do at most 20 times 
    # if i > 2000:
    #   break  

INFO:lightning_fabric.utilities.seed:Global seed set to 60190


Len dataset: 9869
Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [02:51<00:00,  3.43s/it]
INFO:lightning_fabric.utilities.seed:Global seed set to 56178


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [02:51<00:00,  3.42s/it]
INFO:lightning_fabric.utilities.seed:Global seed set to 14572


...

DDIM Sampler: 100%|██████████| 50/50 [02:51<00:00,  3.43s/it]
INFO:lightning_fabric.utilities.seed:Global seed set to 50776


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [02:51<00:00,  3.43s/it]
INFO:lightning_fabric.utilities.seed:Global seed set to 62752


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [02:51<00:00,  3.42s/it]


In [2]:
plt.imshow(results[0])
sample = results[0]

cv2.imwrite("samples/test_sample.png", sample)

img_converter_to_audio = SpectrogramImageConverter(SpectrogramParams(sample_rate=44100, min_frequency=0, max_frequency=10000))

sample_img = Image.open("samples/test_sample.png") 

out_audio_recon = img_converter_to_audio.audio_from_spectrogram_image(sample_img, apply_filters=True).set_channels(2)
out_audio_recon.export("test_sample.wav", format="wav")
out_audio_recon

In [12]:
# listen to target

target = (test_dataset[i]['jpg'] + 1) / 2 * 255
cv2.imwrite("samples/test_sample_target.png", target)
target_img = Image.open("samples/test_sample_target.png") 

out_audio_recon = img_converter_to_audio.audio_from_spectrogram_image(target_img, apply_filters=True).set_channels(2)
out_audio_recon.export("test_sample_target.wav", format="wav")
out_audio_recon